In [2]:
"""
1. Search for an open opportunity at your possible potential clients
2. Search for relevant information from the opening that might act as a good starting point for your business with this company
3. Go through all of your company's projects and find relevant projects
4. Get important to share information about these projects
5. Write a compelling cold email about how your firm can be of great service to your potential client
6. Repeat for ALL potential clients
"""

"\n1. Search for an open opportunity at your possible potential clients\n2. Search for relevant information from the opening that might act as a good starting point for your business with this company\n3. Go through all of your company's projects and find relevant projects\n4. Get important to share information about these projects\n5. Write a compelling cold email about how your firm can be of great service to your potential client\n6. Repeat for ALL potential clients\n"

In [3]:
# Web Scraping

# Relevant information

# Chromadb query

# Email Generation


In [4]:
# Web Scraping
!pipenv install langchain_community

Installing langchain_community...
Installation Succeeded
Installing dependencies from Pipfile.lock (385925)...
All dependencies are now up-to-date!
Building requirements...
[    ] Locking packages...
Resolving dependencies...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[   =] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[====] Locking packages...
[=== ] Locking packages...
[==  ] Locking packages...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[    ] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[====] Locking packages...
[=== ] Locking packages...
[=   ] Locking packages...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking pac

Loading .env environment variables...
Upgrading langchain_community in  dependencies.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://meinestadt.jobs.personio.de/job/2059155?language=en&display=de")
page_data = loader.load().pop().page_content
#print(page_data)

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

#response = llm.invoke("Who is the best footballer of all times?, no preamble")
#print(response.content)


In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = prompt_extract | llm 
response = chain_extract.invoke(input={'page_data':page_data})
print(type(response.content))
print(response.content)


     

<class 'str'>
{
"role": "Manager Business Development & Partnerships",
"experience": "Mind. 3 Jahre Erfahrung im Business Development oder Partnermanagement im digitalen Umfeld",
"skills": ["strategische Partnerschaften", "Business Development", "Partnermanagement", "digitale Geschäftsmodelle", "Excel", "PowerPoint", "Google Slides", "Deutsch", "Englisch"],
"description": "Deine Mission: Partnerschaften & Business Development, Strategische Weiterentwicklung, Stakeholder Management & Kommunikation, Operatives Partnermanagement"
}


In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
print(type(json_response))


<class 'dict'>


In [22]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        for row in csv_reader:
            # Separate technical skills (list) and project link (string)
            skills = tuple(row[:-1])  # Exclude the last element (project link)
            project_link = row[-1]  # Get the last element (project link)
            data.append((skills, project_link))  # Create a tuple with skills and link
    return data

# Example usage:
file_path = "D:\cheddkhani\coldemail\pythonProject1\DiyaManna-Resume.pdf-_3_.csv" # Replace with your actual file path
data = read_csv_file(file_path)

for skills, project_link in data:
    print(skills, project_link)
    #print(project_link)

() COMPUTER SCIENCE STUDENT
() +91 9475396107          diya.manna.ru@gmail.com
() Patia,Bhubaneswar, 751024
() 
() 
() 
() PROFILE
() https://github.com/DiyaManna?
() A dynamic and self-driven individual
() tab=repositories
() looking for an organization where I can enhance my
() https://www.linkedin.com/in/          skills and contribute to the
() diya-manna-310551267/
() organizationâ€™s growth
() 
() 
() 
() TECHNICAL SKILLS
() EXPERIENCE
() Language:
() Content Lead:
() C, C++, Java, Python, HTML/CSS, JavaScript
() KIIT Robotics Society - 2023 - Present
() Web Dev:
() Next.js(React), Bootstrap, Node.                   Social Volunteer
() Databases:                                  NSS Target JEE: 2023- Present
() SQL, MongoDB
() Data Science:
() Tensorflow, Numpy, Pandas, Matplotlib
() PREVIOUS WORK
() DevTools:                                Internship Experience: LINK
() Git, GitHub, VSCode
() Content Writer at Cogent IBS â€“ I was responsible for crafting engaging blogs
() and c

<>:17: SyntaxWarning: invalid escape sequence '\c'
<>:17: SyntaxWarning: invalid escape sequence '\c'
C:\Users\KIIT\AppData\Local\Temp\ipykernel_6192\3051137433.py:17: SyntaxWarning: invalid escape sequence '\c'
  file_path = "D:\cheddkhani\coldemail\pythonProject1\DiyaManna-Resume.pdf-_3_.csv" # Replace with your actual file path


In [23]:
# Insert data into Vector Database

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio_links')

if not collection.count():
    for skills, project_url in data:
        collection.add(
            documents=str(skills),
            metadatas={'portfolio_url':project_url},
            ids=[str(uuid.uuid4())]
        )


In [24]:
json_response['skills']

['strategische Partnerschaften',
 'Business Development',
 'Partnermanagement',
 'digitale Geschäftsmodelle',
 'Excel',
 'PowerPoint',
 'Google Slides',
 'Deutsch',
 'Englisch']

In [25]:
portfolio_urls = collection.query(query_texts=json_response['skills'], n_results=2)
portfolio_urls

{'ids': [['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272'],
  ['467f9ada-2c03-4f7a-8cbd-e4d3c09325e8',
   'c17abb04-75ed-4ee1-ae6a-a8b86675d272']],
 'embeddings': None,
 'documents': [['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()'],
  ['()', '()']],
 'uris': None,
 'included': ['

In [26]:
job_description = json_response['description']

In [27]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Hassan, You are an incredible business development officer who knows how to get clients. You work for X Consulting firm, your firm works with all sorts of IT clients and provide solutions in the domain of Data Science and AI. 
    X AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':job_description, 'portfolio_urls':portfolio_urls})
print(response.content)


Here is a cold email that I came up with:

Subject: Partnership Opportunity for Deine Mission: Partnerschaften & Business Development Team,

Dear [Hiring Manager's Name],

I came across your job posting for Partnerschaften & Business Development, Strategische Weiterentwicklung, Stakeholder Management & Kommunikation, Operatives Partnermanagement, and I was impressed by the scope of responsibilities. As a seasoned business development officer at X Consulting firm, I believe our expertise in Data Science and AI can help you achieve your goals.

Our team has a proven track record of delivering efficient, tailored solutions that keep costs down. I'd love to explore how we can support your mission. 

Take a look at our relevant portfolio URLs:

* COMPUTER SCIENCE STUDENT (Portfolio URL)

These examples demonstrate our capabilities in [specific area of expertise]. I'd be happy to set up a call to discuss how we can help you drive growth and success.

Looking forward to hearing from you,

Has